In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
plt.style.use('ggplot')

In [5]:
os.chdir("C://Users//ivarnieuwepc//Documents//GitHub//Data")

In [8]:
#read all tables
df_emd_world = pd.read_csv("Disaster Data whole world 1960 - 2018.csv", sep = ';',encoding = 'latin-1')
#names
#population
#undernourishment

### INFO FAO OVER LOW & MIDDLE INCOME COUNTRIES (nog omschrijven naar eigen verhaal)
NOTES: Low- and middle-income countries with high exposure are defined as exposed to climate extremes (heat, drought, floods and storms) for more than 66 percent of the time, i.e. more than three years in the period 2011–2016. The estimates in the figure refer to unweighted population average of the prevalence of undernourishment in a sample of 51 low- and middle-income countries with high exposure to climate extremes in 2011–2016, for countries showing different combinations of vulnerabilities identified in Box 9 and for 77 low- and middle-income countries with low exposure to climate extremes. See Annex 2 for more detailed definitions and methodology of the different types of vulnerability to climate variability and extremes.

Stappenplan indelen landen in wereld in dummy cattegorie:

Countries with high exposure to climate extremes

Defined as low- and middle-income countries and territories exposed to climate extremes for at least 66 percent of the time, or more than three out of six years during the most recent subperiod of six consecutive years (2011–2016). There are 51 low- and middle-income countries that meet these criteria.

Countries with low exposure to climate extremes

Defined as low- and middle-income countries and territories exposed to climate extremes for up to 50 percent of the time, or less than four out of six years during the most recent subperiod of six consecutive years (2011–2016). There are 78 low- and middle-income countries that meet these criteria.

Deze variabele aanmaken op basis van EMD Data data. Daarna een regressie analyse doen op basis van gemiddelde undernourishment bij Low exposure en high exposure --> checken of verschil significant is. Dus of climate extremes significant bijdragen aan undernourisment

In [9]:
#renaming environmental hazards dataframe to make it linkable with country information (see below)
df_emd_world = df_emd_world.rename({'Country Code': 'iso','disaster type':'type'}, axis = 1)


#only selecting the four hazards that influence agriculture (as definend by FAO 2017)
types = ["drought","Flood","Extreme temperature", "Storm"]
df_emd_all = df_emd_world[df_emd_world["type"].isin(types)]

#selecting right variables, period 2011 - 2016 and summate some variables per year and country (via iso)
df_emd_all = df_emd_all.groupby(["year", "iso"])[['occurrence','Total deaths','Total affected','Total damage']].sum().reset_index()
df_emd_all = df_emd_all[df_emd_all["year"]<2017]
df_2011_2016 = df_emd_all[df_emd_all["year"]>2010]
df_emd_done = df_2011_2016[["iso","occurrence"]]

In [12]:
#adding additional country information (about low & middle income groups)
country_names = pd.read_csv("country_names.csv", sep = ',')
#dropping high income countries and countries without information about income group
country_names = country_names[country_names["IncomeGroup"]!='High income']
country_names = country_names.dropna(axis=0, subset=['IncomeGroup']) 
#only keeping the columns with the right information
country_names = country_names[["Country Code","Region","IncomeGroup","TableName"]]
country_names = country_names.rename({'Country Code':'iso'}, axis=1)
#merge dataframe about worldwide hazards only keeping low & middle income countries
df_dummy = pd.merge(country_names,df_emd_done, right_on = 'iso',left_on = 'iso', how = 'inner')

#calculating occurrence within period 2011-2016 and adding dummy variable for high exposure ([dummy]=1) and low exposure ([dummy]=0)
df_dummy = df_2011_2016.groupby(['iso'])[["year"]].count().reset_index()
df_dummy['dummy'] = np.where(df_dummy['year']>4, 1, 0)
df_dummy = df_dummy.rename({'year':'occurence'}, axis=1)
#add country info back in 
df_dummy = pd.merge(df_dummy,country_names, left_on = 'iso', right_on = 'iso')

In [118]:
#add undernourishment information (could also be done with whole WDI table)
dfu = pd.read_csv("undernourisment_world.csv", sep = ',')
dfu = pd.melt(dfu, id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], var_name='Years')
dfu = dfu.rename({'Country Code': 'iso'}, axis = 1)
#put it into the dummy dataframe
dfu = pd.merge(df_dummy,dfu,left_on = 'iso', right_on = 'iso')
#only keeping year 2015 for the analysis
dfu2015 = dfu[dfu['Years']=='2015']

#extracting arrays for the independent T-Test Function
low_hazard = dfu2015[dfu2015["dummy"]==0]["value"].as_matrix()
high_hazard = dfu2015[dfu2015["dummy"]==1]["value"].as_matrix()

#performing t-test
t_test = stats.ttest_ind(low_hazardar, high_hazardar, nan_policy = 'omit')

dfpop1 = pd.read_csv("population_conv.csv",sep = ',')
dfpop = pd.melt(dfpop1, id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], var_name='Years')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


dummy
0    10.802381
1    14.083673
Name: value, dtype: float64

In [3]:
#add population info to calculate total undernourished population 
dfpop = pd.read_csv("population_conv.csv",sep = ',')
dfpop = pd.melt(dfpop1, id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], var_name='Years')
dfpop = dfpop.rename({'Country Code': 'iso'}, axis = 1)
dfpop = dfpop[dfpop["Years"]=='2015']
dfpop = dfpop[["iso","value"]]
dfpop = dfpop.rename({'value':'population'}, axis=1)

#combine dataframes
df_total_2015 = pd.merge(dfpop,dfu2015,right_on = 'iso', left_on = 'iso')
df_total_2015 = df_total_2015[["iso","population","dummy","Country Name","value"]]
df_total_2015 = df_total_2015.rename({'value':'PoU'}, axis=1)

FileNotFoundError: File b'population_conv.csv' does not exist

In [130]:
#combine undernourishment & population dataframe for low & middle income countries
dfpopfu2015 = pd.merge(dfpop,dfu2015,right_on = 'iso', left_on = 'iso')
dfpopfu2015 = dfpopfu2015[["iso","population","dummy","Country Name","value"]]
dfpopfu2015 = dfpopfu2015.rename({'value':'PoU'}, axis=1)
#calculate new column for total undernurist population 
dfpopfu2015["abs population undernourist"] = dfpopfu2015.population * (dfpopfu2015.PoU/100)

#calculate barchart values for total absolute population and PoU for countries with high & low exposure to climate extremes
df_total_2015["abs population undernourist"] = df_total_2015.population * (df_total_2015.PoU/100)
absolute_undernourishment = df_total_2015.groupby("dummy")["abs population undernourist"].sum()
average_undernourishment =  (dfu2015.groupby(["dummy"])["value"].sum())/ (dfu2015.groupby(["dummy"])["value"].count())

#PLOT BARCHART HERE